In [1]:
from main.models import Zipcode, BlockGroup, Neighborhood, Listing
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon, 
    MultiPolygon,
    WKBReader, 
    WKBWriter,
    Point
)
from django.contrib.gis.gdal import DataSource
from django.db.models import ExpressionWrapper, F
import re
from lxml import etree

## Discussion regarding overlap detection of unevenly aligned geo boundaries
http://gis.stackexchange.com/questions/69139/how-do-i-use-polygon-centroids-within-a-geoqueryset

## ZCTA Import

In [2]:
# Open US Zip Code Tabulation Areas shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_us_zcta510_500k/cb_2015_us_zcta510_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [3]:
# Create and save zipcode objects
for feature in layer:
    # Extract attrs
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update zipcode object
    Zipcode.objects.update_or_create(
        zipcode=attrs['ZCTA5CE10'],
        land_area=float(attrs['ALAND10']),
        water_area=float(attrs['AWATER10']),
        mpoly=geom
    )

## Block Group Import

In [4]:
# Open Los Angeles County Block Groups shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_06_bg_500k/cb_2015_06_bg_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [5]:
# Create and save BlockGroup objects
for feature in layer:
    # Extract attributes
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    
    # Extract geometry
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update object
    BlockGroup.objects.update_or_create(
        geoid=attrs['GEOID'], 
        land_area=float(attrs['ALAND']), 
        water_area=float(attrs['AWATER']),
        mpoly=geom
    )

In [9]:
# Assign listings to zipcodes
Listing.objects.update(zipcode=None)
for l in Listing.objects.all():
    zipcodes = Zipcode.objects.filter(mpoly__contains_properly=l.point)
    if zipcodes.count() == 1:
        l.zipcode = zipcodes.first()
        l.save()
    else:
        print(l.id, zipcodes, '\n')

287339 <QuerySet []> 

656963 <QuerySet []> 

2227286 <QuerySet []> 

8244781 <QuerySet []> 

6986728 <QuerySet []> 

10638942 <QuerySet []> 

11038162 <QuerySet []> 

11083126 <QuerySet []> 

11362231 <QuerySet []> 

12849655 <QuerySet []> 

12863157 <QuerySet []> 

12866294 <QuerySet []> 

13297945 <QuerySet []> 

13682291 <QuerySet []> 

13811274 <QuerySet []> 

13896738 <QuerySet []> 

13911140 <QuerySet []> 

13936161 <QuerySet []> 

13960487 <QuerySet []> 

14079456 <QuerySet []> 

14282984 <QuerySet []> 



In [19]:
# assign listings to block groups
for l in Listing.objects.all():
    bgs = BlockGroup.objects.filter(mpoly__contains_properly=l.point)
    if bgs.count() == 1:
        l.block_group = bgs.first()
        l.save()
    else:
        print(l.pk, bgs, '\n')

287339 <QuerySet []> 

656963 <QuerySet []> 

2227286 <QuerySet []> 

8244781 <QuerySet []> 

6986728 <QuerySet []> 

10638942 <QuerySet []> 

11038162 <QuerySet []> 

11362231 <QuerySet []> 

12849655 <QuerySet []> 

12863157 <QuerySet []> 

12866294 <QuerySet []> 

13297945 <QuerySet []> 

13682291 <QuerySet []> 

13811274 <QuerySet []> 

13896738 <QuerySet []> 

13911140 <QuerySet []> 

13936161 <QuerySet []> 

13960487 <QuerySet []> 

14079456 <QuerySet []> 

14282984 <QuerySet []> 

